In [ ]:
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

In [ ]:
!pip install gensim

In [5]:
import urllib.request
from gensim.models.word2vec import Word2Vec

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

('ratings.txt', <http.client.HTTPMessage at 0x7f00698a4250>)

In [ ]:
train_data = pd.read_table('ratings.txt')
train_data.head()

In [9]:
train_data = train_data.dropna()

In [52]:
train_data['clean_doc'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z ]", "")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [53]:
stw = pd.read_csv('/content/drive/MyDrive/text_minning/stopwords-ko.txt', header=None)
stw_list = list(stw[0].values)

In [54]:
from konlpy.tag import Mecab

In [55]:
mecab = Mecab()
tokenized_data = []
for sentence  in train_data['clean_doc']:
  temp_x = mecab.morphs(sentence)
  temp_x = [word for word in temp_x if not word in stw_list]
  tokenized_data.append(temp_x)

In [ ]:
tokenized_data

In [57]:
token_count = {}

for line in tokenized_data:
  for tok in line:
    if tok in token_count:
      token_count[tok] += 1
    else:
      token_count[tok] = 1
token_count
len(token_count)

59708

In [58]:
from gensim.models import Word2Vec
model = Word2Vec(sentences = tokenized_data, size=100, window=5, min_count=5, sg=0)

In [59]:
model.wv.vectors.shape

(18255, 100)

In [ ]:
model.wv.most_similar('최민식')

In [ ]:
model.wv.most_similar('무협')

In [34]:
model['무협'].shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


(100,)

In [60]:
from gensim.models import KeyedVectors
model.wv.save_word2vec_format('kor_w2v')

In [61]:
loaded_model = KeyedVectors.load_word2vec_format('kor_w2v')

In [ ]:
loaded_model.most_similar('영화')

In [ ]:
model['비티에스']

In [71]:
loaded_model.most_similar('tree')

KeyError: ignored

In [66]:
from gensim.models import FastText
fasttext_model = FastText(sentences = tokenized_data, size=100, window=5, min_count=5, sg=0)

In [72]:
fasttext_model.most_similar('tree')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('made', 0.947812557220459),
 ('시고니', 0.9457277655601501),
 ('장끌로드', 0.9337199926376343),
 ('장끌로드반담', 0.9321539402008057),
 ('꾸러기', 0.9318756461143494),
 ('인디펜던스', 0.9317868947982788),
 ('개간지', 0.9317042827606201),
 ('초특급', 0.9303422570228577),
 ('Rock', 0.9297130107879639),
 ('trash', 0.9284185171127319)]

In [73]:
# pretrained Embedding

In [1]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding

In [75]:
sample_text1 = 'bitty bought a bit of butter'
sample_text2 = 'but the bit of butter was a bit bitter'
sample_text3 = 'so she bought some better butter to make the bitter butter better'

corp = [sample_text1, sample_text2, sample_text3]
corp

['bitty bought a bit of butter',
 'but the bit of butter was a bit bitter',
 'so she bought some better butter to make the bitter butter better']

In [77]:
vocab_size= 50
encod_corp = []

for i, doc in enumerate(corp):
  encod_corp.append(one_hot(doc, 50))

encod_corp

[[33, 48, 27, 32, 3, 25],
 [32, 11, 32, 3, 25, 17, 27, 32, 19],
 [31, 45, 48, 46, 19, 25, 20, 40, 11, 19, 25, 19]]

In [78]:
model = Sequential()
model.add(Embedding(vocab_size, 128))

In [79]:
embeddings = model.layers[0].get_weights()[0]
embeddings

array([[-0.00426627,  0.02879318,  0.0292148 , ...,  0.04560167,
        -0.0127991 ,  0.00879269],
       [-0.01603134,  0.04371004, -0.00281749, ..., -0.02656709,
         0.02914966,  0.00753963],
       [-0.04890982, -0.02958933, -0.00266454, ..., -0.01192166,
        -0.04480021,  0.04573828],
       ...,
       [ 0.01836227, -0.0030772 , -0.02417002, ...,  0.03665283,
         0.0356665 , -0.03199582],
       [ 0.04988967,  0.04809321,  0.02169574, ...,  0.00783038,
        -0.02878051, -0.03145571],
       [ 0.01335565,  0.00199808,  0.03224312, ..., -0.04302135,
        -0.01987056,  0.0054441 ]], dtype=float32)

In [80]:
embeddings.shape

(50, 128)

In [ ]:
embeddings[30]

In [83]:
# 감성분류

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
sentences = [
             'nice great best amazing',
             'stop lies',
             'pitiful nerd',
             'excellent work',
             'supreme quality',
             'bad',
             'highly respectable'
]

y_train = [1,0,0,1,1,0,1]

In [4]:
t = Tokenizer()
t.fit_on_texts(sentences)

In [ ]:
t.word_index

In [94]:
t.num_words

In [6]:
vocab_size = len(t.word_index) + 1

print(vocab_size)

16


In [9]:
import pandas as pd
import numpy as np

In [31]:
x_encoded = t.texts_to_sequences(sentences)
x_encoded

[[1, 2, 3, 4], [5, 6], [7, 8], [9, 10], [11, 12], [13], [14, 15]]

In [10]:
x_train = pad_sequences(x_encoded, maxlen=4)
y_train = np.array(y_train)

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

model = Sequential()
model.add(Embedding(vocab_size, 4, input_length=4))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [12]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
model.fit(x_train, y_train, epochs=100, verbose=2)

In [106]:
# 구글에서 사전에 훈련된 word2vec을 사용..

In [108]:
!wget "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2022-04-12 03:40:40--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.101.101
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.101.101|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2022-04-12 03:40:40 ERROR 404: Not Found.



In [24]:
import gensim

In [26]:
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/text_minning/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [32]:
word2vec_model.vectors.shape

(3000000, 300)

In [ ]:
# 16 * 300

embedding_matrix = np.zeros((vocab_size, 300))
embedding_matrix

In [35]:
# google w2v에서 해당 토큰의 vector값을 추출..

def get_vector(word):
  if word in word2vec_model:
    return word2vec_model[word]
  else:
    return None

In [ ]:
t.word_index

In [39]:
for word, i in t.word_index.items():
  temp = get_vector(word)
  if temp is not None:
    embedding_matrix[i] = temp

In [41]:
embedding_matrix.shape

(16, 300)

In [43]:
from tensorflow.keras.layers import Input

In [ ]:
model = Sequential()
model.add(Input(shape=(4,), dtype='int32'))
e = Embedding(16, 300, weights=[embedding_matrix], trainable=False) 
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.summary()


In [45]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
model.fit(x_train, y_train, epochs=100, verbose=2)

In [47]:
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow as tf

In [ ]:
(train_data, validation_data, test_data), info = tfds.load('imdb_reviews', split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'], \
                                                                                  with_info=True, as_supervised=True)

In [ ]:
train_example_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_example_batch

In [55]:
train_example_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [53]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [ ]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=False)
hub_layer(train_example_batch[:3])

In [56]:
model = Sequential()
model.add(hub_layer)
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  (None, 20)                400020    
                                                                 
 dense_6 (Dense)             (None, 16)                336       
                                                                 
 dense_7 (Dense)             (None, 1)                 17        
                                                                 
Total params: 400,373
Trainable params: 353
Non-trainable params: 400,020
_________________________________________________________________


In [57]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [58]:
hist = model.fit(train_data.shuffle(10000).batch(512),
                 epochs=20,
                 validation_data=validation_data.batch(512),
                 verbose=1)

Epoch 1/20
40/40 [==============================] - 6s 87ms/step - loss: 0.9572 - acc: 0.4808 - val_loss: 0.7622 - val_acc: 0.5228
Epoch 2/20
40/40 [==============================] - 4s 80ms/step - loss: 0.7079 - acc: 0.5464 - val_loss: 0.6716 - val_acc: 0.5908
Epoch 3/20
40/40 [==============================] - 4s 76ms/step - loss: 0.6556 - acc: 0.6135 - val_loss: 0.6366 - val_acc: 0.6380
Epoch 4/20
40/40 [==============================] - 4s 83ms/step - loss: 0.6341 - acc: 0.6399 - val_loss: 0.6188 - val_acc: 0.6552
Epoch 5/20
40/40 [==============================] - 4s 83ms/step - loss: 0.6220 - acc: 0.6538 - val_loss: 0.6079 - val_acc: 0.6644
Epoch 6/20
40/40 [==============================] - 4s 82ms/step - loss: 0.6130 - acc: 0.6633 - val_loss: 0.5999 - val_acc: 0.6708
Epoch 7/20
40/40 [==============================] - 4s 77ms/step - loss: 0.6047 - acc: 0.6718 - val_loss: 0.5922 - val_acc: 0.6796
Epoch 8/20
40/40 [==============================] - 4s 77ms/step - loss: 0.5976 - a